This notebook was modified from the DSBA-6190 Project 2 notebook to incorporate use of Amazon Sagemaker.

The Wine Quality Dataset (https://archive.ics.uci.edu/ml/datasets/Wine+Quality) was used as the data input. Several regression models were generated, using the provided data to predict red wine quality.

# Import

## Libraries / Packages

In [1]:
# General
import pandas as pd
import numpy as np

import os
import tarfile
import datetime

#SageMaker
import boto3
import io
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, CategoricalParameter

# Data Preparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modeling
from sklearn.ensemble import RandomForestRegressor

# Output
from sklearn.metrics import mean_squared_error,r2_score

# Export
from sklearn.externals import joblib

seed = 5590

## Sagemaker Settings

In [26]:
region = boto3.session.Session().region_name
print('Region: {}'.format(region))

#role = get_execution_role()
role = 'arn:aws:iam::726963482731:role/service-role/AmazonSageMaker-ExecutionRole-20200222T172260'
print('Role: {}'.format(role))

sm_boto3 = boto3.client('sagemaker')
print(sm_boto3)

Region: us-east-1
Role: arn:aws:iam::726963482731:role/service-role/AmazonSageMaker-ExecutionRole-20200222T172260


## Data

Initialize the S3 connection and name the target bucket.

In [3]:
'''
s3_client = boto3.client('s3')
data_bucket_name='dsba-6190-final-team-project'
'''

Check the contents of the bucket, and assign the import data file path and name to the variable **file_data**.

In [7]:
'''
obj_list=s3_client.list_objects(Bucket=data_bucket_name)
'''

In [9]:
'''
obj_list['Contents']
'''

[{'Key': 'build-test/',
  'LastModified': datetime.datetime(2020, 3, 2, 21, 2, 14, tzinfo=tzlocal()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': 'data/',
  'LastModified': datetime.datetime(2020, 2, 22, 22, 44, 56, tzinfo=tzlocal()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': 'data/winequality-red.csv',
  'LastModified': datetime.datetime(2020, 2, 22, 22, 45, 57, tzinfo=tzlocal()),
  'ETag': '"2daeecee174368f8a33b82c8cccae3a5"',
  'Size': 84199,
  'StorageClass': 'STANDARD'},
 {'Key': 'driver_imgs_list.csv',
  'LastModified': datetime.datetime(2020, 3, 2, 2, 39, 1, tzinfo=tzlocal()),
  'ETag': '"a4640e8465d32fa54358c06cb642ca87"',
  'Size': 491359,
  'StorageClass': 'STANDARD'},
 {'Key': 'imgs/test/img_1.jpg',
  'LastModified': datetime.datetime(2020, 3, 2, 2, 39, 3, tzinfo=tzlocal()),
  'ETag': '"f87f22b332c6424414fe3e739a0cf413"',
  'Size': 41638,
  'StorageClass': 'STA

In [20]:
'''
obj_list=s3_client.list_objects(Bucket=data_bucket_name)
file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
#print(file)
file_data=file[2]
'''

In [33]:
'''
file_data
'''

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-33-3090a6019626>, line 2)

Import the data into the variable **response_body** then read into a dataframe.

In [24]:
'''
response = s3_client.get_object(Bucket=data_bucket_name, Key=file_data)
print(list(response))
response_body = response['Body'].read()
df_wine_red = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=";", low_memory=False) 
'''

['ResponseMetadata', 'AcceptRanges', 'LastModified', 'ContentLength', 'ETag', 'ContentType', 'Metadata', 'Body']


# Preprocessing

We need to define input file as a string to the S3 bucket location for the SKLearn process to be able to call.

In [11]:
# Input File Path Check
input_data = 's3://dsba-6190-final-team-project/data/winequality-red.csv'.format(region)

## Preprocessing

In [27]:
#Initialize SKLearn Preprocessing Function
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role, 
                                     base_job_name = "scikit-preprocessing",
                                     instance_type='ml.m4.xlarge',
                                     instance_count=1)

### Preprocessing Script
The following script is used to run the preproocessing stage.

In [13]:
%%writefile preprocessing.py

import argparse
import os
import warnings

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

seed = 5590

if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type=float, default=0.2)
    args, _ = parser.parse_known_args()
    
    print('Received arguments {}'.format(args))
    
    input_data_path = os.path.join('/opt/ml/processing/input', 'winequality-red.csv')
    
    print('Reading input data from {}'.format(input_data_path))
    df = pd.read_csv(input_data_path, header=0, delimiter=";", low_memory=False)
    
      
    split_ratio = args.train_test_split_ratio
    
    print('Splitting data into train and test sets with ratio {}'.format(split_ratio))
    
    #Split Data into training / test
    X_train, X_test, y_train, y_test = train_test_split(df.drop('quality', axis=1), 
                                                        df['quality'], 
                                                        test_size=split_ratio, 
                                                        random_state=seed)
    
    # Transform Features
    scaler = StandardScaler().fit(X_train)
    train_features = scaler.transform(X_train)
    test_features = scaler.transform(X_test)
    
    # Convert Data to Float32
    train_features = train_features.astype('float32')
    test_features = test_features.astype('float32')
    
    # Create local output directories
    try:
        os.makedirs('/opt/ml/processing/output/train')
        os.makedirs('/opt/ml/processing/output/test')
    except:
        pass
    
    print('Train Features shape after preprocessing: {}'.format(train_features.shape))
    print('Train Labels shape after preprocessing: {}'.format(y_train.shape))
    print('Test Features shape after preprocessing: {}'.format(test_features.shape))
    print('Test Labels shape after preprocessing: {}'.format(y_test.shape))
    
    train_features_output_path = os.path.join('/opt/ml/processing/output/train', 'train_features.csv')
    train_labels_output_path = os.path.join('/opt/ml/processing/output/train', 'train_labels.csv')
    
    test_features_output_path = os.path.join('/opt/ml/processing/output/test', 'test_features.csv')
    test_labels_output_path = os.path.join('/opt/ml/processing/output/test', 'test_labels.csv')
    
    print('Saving training features to {}'.format(train_features_output_path))
    pd.DataFrame(train_features).to_csv(train_features_output_path, header= False, index=False)
    
    print('Saving test features to {}'.format(test_features_output_path))
    pd.DataFrame(test_features).to_csv(test_features_output_path, header=False, index=False)
    
    print('Saving training labels to {}'.format(train_labels_output_path))
    y_train.to_csv(train_labels_output_path, header=False, index=False)
    
    print('Saving test labels to {}'.format(test_labels_output_path))
    y_test.to_csv(test_labels_output_path, header=False, index=False)

Writing preprocessing.py


The preprocessing script can now be run with the SKLearnProcessor.run() method. 

In [14]:
hyperparameters = {
    'train-test-split-ratio': 0.25
}

In [15]:
hyperparameters

{'train-test-split-ratio': 0.25}

In [32]:
sklearn_processor.run(code='preprocessing.py',
                      arguments=['--train-test-split-ratio', '0.3'],
                      inputs=[ProcessingInput(
                        source=input_data,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/output/train'),
                               ProcessingOutput(output_name='test_data',
                                                source='/opt/ml/processing/output/test')]
                     )

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'train_data':
        preprocessed_training_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'test_data':
        preprocessed_test_data = output['S3Output']['S3Uri']

TypeError: run() got an unexpected keyword argument 'base_job_name'

### Check Preprocessing Outputs

In [ ]:
#output_config

Check to see that the training features were processed.

In [30]:
'''
training_features = pd.read_csv(preprocessed_training_data + '/train_features.csv', nrows=5,  header=None)
print('Training features shape: {}'.format(training_features.shape))
training_features
'''

"\ntraining_features = pd.read_csv(preprocessed_training_data + '/train_features.csv', nrows=5,  header=None)\nprint('Training features shape: {}'.format(training_features.shape))\ntraining_features\n"

In [18]:
'''
training_labels = pd.read_csv(preprocessed_training_data + '/train_labels.csv', nrows=5,  header=None)
print('Training labels shape: {}'.format(training_labels.shape))
training_labels
'''

Training labels shape: (5, 1)


,0
0,7
1,5
2,5
3,6
4,7


In [31]:
'''
X_train = pd.read_csv(preprocessed_training_data + '/train_features.csv', header=None)

#Checks are for previous inconsistencies in data shape
print('The Training Data Features Dataframe has the following shape: {}'.format(X_train.shape))
print('The Training Data Features Numpy Array has the following shape: {}'.format(np.array(X_train).shape))

X_train.head(5)
'''

"\nX_train = pd.read_csv(preprocessed_training_data + '/train_features.csv', header=None)\n\n#Checks are for previous inconsistencies in data shape\nprint('The Training Data Features Dataframe has the following shape: {}'.format(X_train.shape))\nprint('The Training Data Features Numpy Array has the following shape: {}'.format(np.array(X_train).shape))\n\nX_train.head(5)\n"

In [20]:
'''
y_train = pd.read_csv(preprocessed_training_data + '/train_labels.csv', header=None)

#Checks are for previous inconsistencies in data shape
print('The Training Data Labels Dataframe has the following shape: {}'.format(y_train.shape))
print('The Training Data Labels Numpy Array has the following shape: {}'.format(np.array(y_train).ravel().shape))

y_train.head(5)
'''

The Training Data Labels Dataframe has the following shape: (1119, 1)
The Training Data Labels Numpy Array has the following shape: (1119,)


,0
0,7
1,5
2,5
3,6
4,7


## Training

### Training Script

In [57]:
%%writefile train.py

import argparse
import os

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

from sklearn.model_selection import cross_val_score

seed = 5590

# Inference Function
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__=="__main__":
    
    print('Extracting Arguments')
    parser = argparse.ArgumentParser()
    
    # Hyperparameters
    parser.add_argument('--n-estimators', type=int, default=750)
    parser.add_argument('--max-features', type=str, default='sqrt')
    
    # Directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file_features', type=str, default = 'train_features.csv')
    parser.add_argument('--train_file_labels', type=str, default = 'train_labels.csv')
    parser.add_argument('--test_file_features', type=str, default = 'test_features.csv')
    parser.add_argument('--test_file_labels', type=str, default = 'test_labels.csv')
    
    args = parser.parse_args()
    
    # Data Paths
    train_features_data = os.path.join(args.train, args.train_file_features)
    train_labels_data = os.path.join(args.train, args.train_file_labels)
    
    #test_features_data = os.path.join(args.test, args.test_file_features)
    #test_labels_data = os.path.join(args.test, args.test_file_labels)
    
    
    print('Reading input data')
    # Import Data
    ## Training
    X_train = pd.read_csv(train_features_data)
    y_train = pd.read_csv(train_labels_data)
    
    ## Test
    #X_test = pd.read_csv(test_features_data)
    #y_test = pd.read_csv(test_labels_data)
    
    # Convert Data to NP Array
    ## Training
    X_train = np.array(X_train)
    y_train = np.array(y_train).ravel()
    
    ## Test
    #X_test = np.array(X_test)
    #y_test = np.array(y_test).ravel()
    
    # Check Data Shape
    print(X_train.shape)
    print(y_train.shape)
    
    # Initialize Random Forest Regressor Model. 
    # Hyperparameters based on earlier local analysis
        
    model = RandomForestRegressor(n_estimators = args.n_estimators,
                                  max_features = args.max_features,
                                  random_state = seed)
    
    print('Training Random Forest Regression model')
    model.fit(X_train, y_train)
    
    print('Training Complete')
    
    # Metrics
    
    mse_neg = cross_val_score(model, X_train, y_train, scoring = 'neg_mean_squared_error', cv=10)
    mse_abs = np.absolute(mse_neg)
    rsme = np.sqrt(mse_abs)
    rsme_avg = np.mean(rsme)
    print('RMSE: {}'.format(rsme_avg))
    
    model_output_directory = os.path.join(args.model_dir, "model.joblib")
    print('Saving model to {}'.format(model_output_directory))
    joblib.dump(model, model_output_directory)

Overwriting train.py


### Create SKLearn Estimator For Training

In [58]:
sklearn_est = SKLearn(
    entry_point='train.py',
    role=role,
    train_instance_count=1,
    train_instance_type="ml.c5.xlarge",
    framework_version='0.20.0',
    base_job_name='red-wine-rf-scikit-training',
    metric_definitions=[
        {'Name': 'RSME',
        'Regex': "RSME: ([0-9.]+).*$"}],
    hyperparameters = {'n-estimators': 500}
)

### Fit SKLearn Esitmator to Training Data

In [59]:
sklearn_est.fit(
    {
        'train': preprocessed_training_data, 
        'test': preprocessed_test_data
    },
    wait = False
)

### Training Job Output Information

In [60]:
training_job_description = sklearn_est.jobs[-1].describe()

model_data_s3_uri = '{}{}/{}'.format(
    training_job_description['OutputDataConfig']['S3OutputPath'],
    training_job_description['TrainingJobName'],
    'output/model.tar.gz')

## Hyperparameter Tuning Test

In [67]:
'''
seed = 5590

# Define exploration boundaries
hyperparameter_ranges = {
    'n-estimators': IntegerParameter(10, 1000),
    'max-features': CategoricalParameter(['auto', 'sqrt', 'log2'])}

# create Optimizer
Optimizer = sagemaker.tuner.HyperparameterTuner(
    estimator=sklearn_est,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name='red-wine-rf-hpo',
    objective_type='Minimize',
    objective_metric_name='RMSE',
    metric_definitions=[
        {'Name': 'RMSE',
        'Regex': "RMSE: ([0-9.]+).*$"}],
    max_jobs=15,
    max_parallel_jobs=3)
'''

In [68]:
'''
Optimizer.fit(
    {
        'train': preprocessed_training_data, 
        'test': preprocessed_test_data
    },
    wait = False
)
'''

In [84]:
'''
tuner = Optimizer.analytics()
df_tune_ouput_all = tuner.dataframe()

df = df_tune_ouput_all[df_tune_ouput_all['FinalObjectiveValue'] > -float('inf')]

df
'''

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,max-features,n-estimators
0,0.598190,55.0,2020-02-29 17:50:30+00:00,red-wine-rf-hpo-200229-1732-015-60904f48,Completed,2020-02-29 17:49:35+00:00,"""log2""",153.0
1,0.600965,41.0,2020-02-29 17:48:52+00:00,red-wine-rf-hpo-200229-1732-014-c78d88a4,Completed,2020-02-29 17:48:11+00:00,"""log2""",89.0
2,0.608505,54.0,2020-02-29 17:49:45+00:00,red-wine-rf-hpo-200229-1732-013-3a0ecf82,Completed,2020-02-29 17:48:51+00:00,"""log2""",28.0
3,0.597289,84.0,2020-02-29 17:47:40+00:00,red-wine-rf-hpo-200229-1732-012-00dbcba5,Completed,2020-02-29 17:46:16+00:00,"""sqrt""",665.0
4,0.597413,70.0,2020-02-29 17:45:54+00:00,red-wine-rf-hpo-200229-1732-011-1fcf2b15,Completed,2020-02-29 17:44:44+00:00,"""log2""",1000.0
5,0.597129,80.0,2020-02-29 17:45:58+00:00,red-wine-rf-hpo-200229-1732-010-3eab2de1,Completed,2020-02-29 17:44:38+00:00,"""log2""",706.0
6,0.597207,90.0,2020-02-29 17:43:14+00:00,red-wine-rf-hpo-200229-1732-009-c1f213f3,Completed,2020-02-29 17:41:44+00:00,"""log2""",919.0
7,0.597364,63.0,2020-02-29 17:42:16+00:00,red-wine-rf-hpo-200229-1732-008-ac409c29,Completed,2020-02-29 17:41:13+00:00,"""sqrt""",259.0
8,0.597138,68.0,2020-02-29 17:42:42+00:00,red-wine-rf-hpo-200229-1732-007-f82de863,Completed,2020-02-29 17:41:34+00:00,"""sqrt""",704.0
9,0.596790,52.0,2020-02-29 17:38:45+00:00,red-wine-rf-hpo-200229-1732-006-46f3de97,Completed,2020-02-29 17:37:53+00:00,"""log2""",302.0


# Deploy Model

In [89]:
sklearn_est.latest_training_job.wait(logs='None')


2020-02-29 15:06:27 Starting - Preparing the instances for training
2020-02-29 15:06:27 Downloading - Downloading input data
2020-02-29 15:06:27 Training - Training image download completed. Training in progress.
2020-02-29 15:06:27 Uploading - Uploading generated training model
2020-02-29 15:06:27 Completed - Training job completed


In [90]:
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_est.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)

Model artifact persisted at s3://sagemaker-us-east-1-726963482731/red-wine-rf-scikit-training-2020-02-29-15-03-44-768/output/model.tar.gz


In [91]:
sklearn_predictor = sklearn_est.deploy(initial_instance_count=1, 
                                   instance_type='ml.m4.xlarge')

---------------!

# Shut Down Endopoint

In [93]:
'''
sagemaker.Session().delete_endpoint(sklearn_predictor.endpoint)
'''